# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 20 2022 1:32PM,242345,19,ADV80004323,"AdvaGen Pharma, Ltd",Released
1,Jul 20 2022 1:32PM,242345,19,ADV80004324,"AdvaGen Pharma, Ltd",Released
2,Jul 20 2022 1:32PM,242345,19,ADV80004325,"AdvaGen Pharma, Ltd",Released
3,Jul 20 2022 1:32PM,242345,19,ADV80004326,"AdvaGen Pharma, Ltd",Released
4,Jul 20 2022 1:32PM,242345,19,ADV80004327,"AdvaGen Pharma, Ltd",Released
5,Jul 20 2022 1:32PM,242345,19,ADV80004328,"AdvaGen Pharma, Ltd",Released
6,Jul 20 2022 1:32PM,242345,19,ADV80004329,"AdvaGen Pharma, Ltd",Released
7,Jul 20 2022 1:32PM,242345,19,ADV80004330,"AdvaGen Pharma, Ltd",Released
8,Jul 20 2022 1:32PM,242345,19,ADV80004331,"AdvaGen Pharma, Ltd",Released
9,Jul 20 2022 1:32PM,242345,19,ADV80004332,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,242340,Released,1
35,242342,Released,1
36,242343,Released,12
37,242344,Released,8
38,242345,Released,45


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242340,NaN,1.0
242342,NaN,1.0
242343,NaN,12.0
242344,NaN,8.0
242345,NaN,45.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242224,1.0,0.0
242231,1.0,0.0
242233,0.0,19.0
242252,0.0,1.0
242263,0.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242224,1,0
242231,1,0
242233,0,19
242252,0,1
242263,0,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242224,1,0
1,242231,1,0
2,242233,0,19
3,242252,0,1
4,242263,0,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242224,1,
1,242231,1,
2,242233,,19
3,242252,,1
4,242263,,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 20 2022 1:32PM,242345,19,"AdvaGen Pharma, Ltd"
45,Jul 20 2022 1:28PM,242344,10,Eywa Pharma Inc.
53,Jul 20 2022 1:16PM,242343,10,"Methapharm, Inc."
65,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC"
66,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc."
67,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc."
68,Jul 20 2022 12:31PM,242336,12,Hush Hush
69,Jul 20 2022 12:30PM,242335,10,MedStone Pharma LLC
80,Jul 20 2022 12:30PM,242334,20,Alumier Labs Inc.
89,Jul 20 2022 12:28PM,242333,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 20 2022 1:32PM,242345,19,"AdvaGen Pharma, Ltd",,45
1,Jul 20 2022 1:28PM,242344,10,Eywa Pharma Inc.,,8
2,Jul 20 2022 1:16PM,242343,10,"Methapharm, Inc.",,12
3,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC",,1
4,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",,1
5,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc.",,1
6,Jul 20 2022 12:31PM,242336,12,Hush Hush,,1
7,Jul 20 2022 12:30PM,242335,10,MedStone Pharma LLC,,11
8,Jul 20 2022 12:30PM,242334,20,Alumier Labs Inc.,,9
9,Jul 20 2022 12:28PM,242333,10,Emerginnova,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 20 2022 1:32PM,242345,19,"AdvaGen Pharma, Ltd",45,
1,Jul 20 2022 1:28PM,242344,10,Eywa Pharma Inc.,8,
2,Jul 20 2022 1:16PM,242343,10,"Methapharm, Inc.",12,
3,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC",1,
4,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",1,
5,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc.",1,
6,Jul 20 2022 12:31PM,242336,12,Hush Hush,1,
7,Jul 20 2022 12:30PM,242335,10,MedStone Pharma LLC,11,
8,Jul 20 2022 12:30PM,242334,20,Alumier Labs Inc.,9,
9,Jul 20 2022 12:28PM,242333,10,Emerginnova,6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 20 2022 1:32PM,242345,19,"AdvaGen Pharma, Ltd",45,
1,Jul 20 2022 1:28PM,242344,10,Eywa Pharma Inc.,8,
2,Jul 20 2022 1:16PM,242343,10,"Methapharm, Inc.",12,
3,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC",1,
4,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 20 2022 1:32PM,242345,19,"AdvaGen Pharma, Ltd",45.0,NaN
1,Jul 20 2022 1:28PM,242344,10,Eywa Pharma Inc.,8.0,NaN
2,Jul 20 2022 1:16PM,242343,10,"Methapharm, Inc.",12.0,NaN
3,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC",1.0,NaN
4,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 20 2022 1:32PM,242345,19,"AdvaGen Pharma, Ltd",45.0,0.0
1,Jul 20 2022 1:28PM,242344,10,Eywa Pharma Inc.,8.0,0.0
2,Jul 20 2022 1:16PM,242343,10,"Methapharm, Inc.",12.0,0.0
3,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC",1.0,0.0
4,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2907801,84.0,4.0
12,242336,1.0,0.0
16,969164,4.0,0.0
18,242224,0.0,1.0
19,2423034,54.0,3.0
20,726869,29.0,12.0
21,969299,4.0,0.0
22,242329,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2907801,84.0,4.0
1,12,242336,1.0,0.0
2,16,969164,4.0,0.0
3,18,242224,0.0,1.0
4,19,2423034,54.0,3.0
5,20,726869,29.0,12.0
6,21,969299,4.0,0.0
7,22,242329,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,84.0,4.0
1,12,1.0,0.0
2,16,4.0,0.0
3,18,0.0,1.0
4,19,54.0,3.0
5,20,29.0,12.0
6,21,4.0,0.0
7,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,84.0
1,12,Released,1.0
2,16,Released,4.0
3,18,Released,0.0
4,19,Released,54.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21,22
Status,,,,,,,,
Executing,4.0,0.0,0.0,1.0,3.0,12.0,0.0,0.0
Released,84.0,1.0,4.0,0.0,54.0,29.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21,22
0,Executing,4.0,0.0,0.0,1.0,3.0,12.0,0.0,0.0
1,Released,84.0,1.0,4.0,0.0,54.0,29.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21,22
0,Executing,4.0,0.0,0.0,1.0,3.0,12.0,0.0,0.0
1,Released,84.0,1.0,4.0,0.0,54.0,29.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()